# Word-Level 번역기 만들기(Neural Machine Translation (seq2seq) Tutorial)

>- __전공 : 컴퓨터전자시스템 공학부__
>- __학번:  201600765__
>- __이름 : 김주원__

- model.load()를 이용하여 학습된 모델을 불러와서 테스트를 진행해본 실습 파일입니다. 아래와 같이 load_model을 통해 학습시간, 모델 설계 및 재설계 시간을 줄일 수 있습니다.1,2번에 대한 답안은 "실습과제5_Word_Level_Translator_모델_불러오기_1,2번_답안_201600765_김주원.pdf"파일을 참고해주시면 됩니다. 감사합니다.

## 1. 데이터 로드 및 전처리
- 필요한 도구들을 import 

In [1]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

#### 데이터를 로드
- fra.txt 데이터 구조 : 왼쪽의 영어 문장과 오른쪽의 프랑스어 문장 사이에 탭으로 구분되는 구조.

- 이번 챕터에서는 총 33,000개의 샘플을 사용할 예정

In [2]:
num_samples = 33000

http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

- 전처리 함수 구현

In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [4]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

- 구현한 전처리 함수들을 임의의 문장을 입력으로 테스트

In [5]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


#### 모든 전처리를 수행하는 함수 구현
- 전체 데이터에서 33,000개의 샘플불러옴. 
- 훈련 과정에서 교사 강요(Teacher Forcing)을 사용할 예정=> 훈련 시 사용할 디코더의 입력 시퀀스와 실제값에 해당되는 출력 시퀀스를 따로 분리하여 저장
- 입력 시퀀스에는 시작을 의미하는 토큰인 `<sos>`를 추가 
- 출력 시퀀스에는 종료를 의미하는 토큰인`<eos>`를 추가

In [6]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("fra.txt", "r", encoding="utf-8") as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

- 인코더의 입력, 디코더의 입력, 디코더의 실제값을 상위 5개 샘플만 출력

In [7]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
[['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.']]
[['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>']]


- 케라스 토크나이저를 통해 단어 집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩

In [8]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

- 패딩을 수행

In [9]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

- 데이터의 크기(shape)를 확인
- 샘플은 총 33,000개 존재하며 영어 문장의 길이는 8, 프랑스어 문장의 길이 16
- 단어 집합의 크기를 정의
- 단어 집합의 크기는 각각 4,647개와 8,022개

In [10]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4606, 프랑스어 단어 집합의 크기 : 8107


- 단어로부터 정수를 얻는 딕셔너리 구현
- 정수로부터 단어를 얻는 딕셔너리 구현 
  - 훈련을 마치고 예측 과정과 실제값과 결과를 비교하는 경우에 사용

In [11]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

- 테스트 데이터를 분리하기 전에, 적절한 분포를 갖도록 데이터를 섞어주는 과정을 진행
  - 이를 위해서 우선 순서가 섞인 정수 시퀀스 리스트 정의

In [12]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[32250 16672 23036 ...     1    54 21477]


- 데이터셋의 순서로 지정, 샘플들이 기존 순서와 다른 순서

In [13]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [14]:
encoder_input[30997]

array([  3,  20,  44, 126,   1,   0,   0,   0])

In [15]:
decoder_input[30997]

array([  2,  19,  34,  57, 448,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

In [16]:
decoder_target[30997]

array([ 19,  34,  57, 448,   1,   3,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

- 테스트 데이터 : 33,000개의 10%에 해당되는 3,300개의 데이터를 테스트 데이터로 사용

In [17]:
n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [18]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

- 훈련 데이터와 테스트 데이터의 크기(shape)를 출력

In [19]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


## 2. 기계번역어 만들기

- load_model함수를 통해 "Word_level_translator.h5"학습모델 파일 로드해서 코드 한줄로 학습데이터를 불러옴

In [21]:
from tensorflow.keras.models import load_model
model = load_model("Word_level_translator.h5")

- 모델의 파라미터를 확인

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     230300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     405350      input_2[0][0]                    
______________________________________________________________________________________________

## 3. seq2seq 기계 번역기 동작시키기

- seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다름
  -  따라서 테스트 과정을 위해 재설계된 모델을 불러옴

In [27]:
encoder_model = load_model("encoder_model.h5")

- 디코더 정의

In [29]:
decoder_model = load_model("decoder_model.h5")

- 테스트 과정에서의 동작을 위한 decode_sequence 함수 설계

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

- 결과 확인을 위한 함수

In [31]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    sentence = ''
    for i in input_seq:
        if(i!=0):
            sentence = sentence + index_to_src[i]+' '
    return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    sentence =''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[i] + ' '
    return sentence

#### 학번 맨 뒤 2자리를 이용한 인덱스 리스트 생성
- 201600765가 학번이므로 65만이용해서 리스트 생성

In [32]:
StudentID = []
for i in range(1,11):
    StudentID.append(int(str(i)+"65"))
StudentID

[165, 265, 365, 465, 565, 665, 765, 865, 965, 1065]

- 훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력

In [33]:
for seq_index in StudentID:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[1:-5])
  print("\n")

원문 :  tom shifted gears . 
번역문 : tom a change de rapport . 
예측문 : tom a fait sa maison . 


원문 :  is he still here ? 
번역문 : est il encore ici ? 
예측문 : est il la voiture ? 


원문 :  i got a bad grade . 
번역문 : je me suis tape une mauvaise note . 
예측문 : j ai eu une journee . 


원문 :  how was work today ? 
번역문 : comment etait le travail aujourd hui ? 
예측문 : comment etait ce qu il en travail ? 


원문 :  i finally won . 
번역문 : j ai fini par l emporter . 
예측문 : j ai fini . 


원문 :  did you read them ? 
번역문 : les as tu lus ? 
예측문 : les avez vous les ? 


원문 :  take mine . 
번역문 : prenez la mienne . 
예측문 : prenez le mien . 


원문 :  this might hurt . 
번역문 : ca pourrait faire mal . 
예측문 : il peut etre fait de la tete . 


원문 :  they released tom . 
번역문 : ils ont relache tom . 
예측문 : ils ont tom . 


원문 :  tom looks deranged . 
번역문 : tom a l air perturbe . 
예측문 : tom a l air . 




- 테스트 데이터에서 임의로 선택한 인덱스의 샘플의 결과출력

In [36]:
for seq_index in StudentID:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[1:-5])
  print("\n")

원문 :  make an offer . 
번역문 : fais une offre . 
예측문 : fais une en train de tom ! 


원문 :  i ll sue you . 
번역문 : je te ferai un proces . 
예측문 : je vous ferai un te choix . 


원문 :  you are very rich . 
번역문 : vous etes tres riche . 
예측문 : vous etes tres riche . 


원문 :  tom just smiled . 
번역문 : tom venait de rire . 
예측문 : tom a presque l air fou . 


원문 :  come with us . 
번역문 : venez avec nous . 
예측문 : venez avec nous . 


원문 :  buy it . 
번역문 : achete la ! 
예측문 : faites le . 


원문 :  she is a beginner . 
번역문 : elle est novice . 
예측문 : elle est moi . 


원문 :  how much is that ? 
번역문 : combien cela coute t il ? 
예측문 : combien ca ? 


원문 :  that s a pencil . 
번역문 : c est un crayon . 
예측문 : c est un une raison . 


원문 :  is your mom home ? 
번역문 : ta mere est elle a la maison ? 
예측문 : est ce votre mere est elle chez elle ? 


